In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-06 03:07:13.883556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-06 03:07:13.883623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-06 03:07:13.887248: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-06 03:07:13.908494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 03:07:16.646427: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../HAM10000/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/10015
[INFO]: Processed 2000/10015
[INFO]: Processed 3000/10015
[INFO]: Processed 4000/10015
[INFO]: Processed 5000/10015
[INFO]: Processed 6000/10015
[INFO]: Processed 7000/10015
[INFO]: Processed 8000/10015
[INFO]: Processed 9000/10015
[INFO]: Processed 10000/10015
(10015, 224, 224, 3)
(10015,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-06 03:12:11.197585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


313/313 [==============================] - 360s 1s/step
(10015, 62720)


In [9]:
data = features_flatten

In [10]:
num_classes = len(np.unique(labels))

In [11]:
import gc
import os

def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [12]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8093812375249501
Sensitivity: 0.6303198866098654
Specificity: 0.9516253098866516
AUC-ROC: 0.9477169006534147
MCC: 0.6315407357339117
Precision: 0.7562479330891663
F1 Score: 0.6751662816289244

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8073852295409182
Sensitivity: 0.634832632167387
Specificity: 0.9467383407158014
AUC-ROC: 0.9484826241111124
MCC: 0.61671577796735
Precision: 0.7418369721112503
F1 Score: 0.679184518996995

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.7982017982017982
Sensitivity: 0.6080759154469376
Specificity: 0.9428717220900793
AUC-ROC: 0.9335943589853526
MCC: 0.5967612379535598
Precision: 0.6755684330796737
F1 Score: 0.6366234414217218

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7952047952047953
Sensitivity: 0.574355082690425
Specificity: 0.9455672213301856
AUC-ROC: 0.9371271468592443
MCC: 0.5960127521462067
Precision: 0.6262630051327509
F1 Score: 0.5979776713678551

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8131868131868132
Sensitivity: 0.580812947857191
Specificity: 0.9511531557903503
AUC-ROC: 0.9420914467917635
MCC: 0.6334329329903341
Precision: 0.6937334892669507
F1 Score: 0.6146304383287854


In [13]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.5578842315369261
Sensitivity: 0.45141540397403934
Specificity: 0.9179655019184575
AUC-ROC: 0.693659958612071
MCC: 0.32677733434761097
Precision: 0.3328657757667398
F1 Score: 0.3335401316334024

[INFO] Fold 2 / 10 for NB
Accuracy: 0.5718562874251497
Sensitivity: 0.4536520952299204
Specificity: 0.9218514653866581
AUC-ROC: 0.6997854181763559
MCC: 0.34088862190675295
Precision: 0.3425997820269258
F1 Score: 0.34806622731089637

[INFO] Fold 3 / 10 for NB
Accuracy: 0.5738522954091816
Sensitivity: 0.44812052658534535
Specificity: 0.9208428281168294
AUC-ROC: 0.7006317159850244
MCC: 0.3335327464769076
Precision: 0.34229248222685016
F1 Score: 0.33742251604759593

[INFO] Fold 4 / 10 for NB
Accuracy: 0.562874251497006
Sensitivity: 0.3741448003896267
Specificity: 0.919655020155341
AUC-ROC: 0.6620167177543543
MCC: 0.3125325253147558
Precision: 0.2824579379452448
F1 Score: 0.2822036805171658

[INFO] Fold 5 / 10 for NB
Accuracy: 0.5788423153692615
Sensitivity: 0.4

In [14]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.7385229540918163
Sensitivity: 0.348768568491383
Specificity: 0.9235349031731931
AUC-ROC: 0.7780199952761999
MCC: 0.45492362685285137
Precision: 0.5524960650576414
F1 Score: 0.38656988474379494

[INFO] Fold 2 / 10 for KNN


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7075848303393214
Sensitivity: 0.3217620822098434
Specificity: 0.9144192874817606
AUC-ROC: 0.8203747630076387
MCC: 0.38461659849319885
Precision: 0.372145738413502
F1 Score: 0.33640181833369526

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.717564870259481
Sensitivity: 0.3471895034794822
Specificity: 0.9158123354533975
AUC-ROC: 0.7795602969466592
MCC: 0.40918427585399614
Precision: 0.45072342027229245
F1 Score: 0.36972998999386775

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.7295409181636726
Sensitivity: 0.3622835565705251
Specificity: 0.915948049583143
AUC-ROC: 0.7690981529124834
MCC: 0.4267891531985018
Precision: 0.6629556175481522
F1 Score: 0.40527564565904955

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7305389221556886
Sensitivity: 0.37760828623550763
Specificity: 0.9170019403555071
AUC-ROC: 0.8057746099395199
MCC: 0.42627037709373095
Precision: 0.6067152523086253
F1 Score: 0.43191263454643797

[INFO] Fold 6 / 10 for KNN
Accuracy: 0.7172827172827173
Sensitivity: 0.3344966338030311
Speci

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7112887112887113
Sensitivity: 0.3238340908724226
Specificity: 0.9128141863423226
AUC-ROC: 0.7822781150306638
MCC: 0.3827809083306192
Precision: 0.37125632944292114
F1 Score: 0.34102279786971784

[INFO] Fold 9 / 10 for KNN
Accuracy: 0.7142857142857143
Sensitivity: 0.3300937445103308
Specificity: 0.9171044615538934
AUC-ROC: 0.7945488238337356
MCC: 0.40257095400460247
Precision: 0.4378323950113046
F1 Score: 0.35241134921517403

[INFO] Fold 10 / 10 for KNN
Accuracy: 0.7442557442557443
Sensitivity: 0.35866673653187514
Specificity: 0.922235129517861
AUC-ROC: 0.8214499511500765
MCC: 0.46069679331505503
Precision: 0.5624706889751389
F1 Score: 0.3898316591662173


In [15]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6936127744510978
Sensitivity: 0.1930519796191438
Specificity: 0.8795514458555911
AUC-ROC: 0.8513636027396191
MCC: 0.24858469022404392
Precision: 0.408459179988355
F1 Score: 0.19987535474159843

[INFO] Fold 2 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6936127744510978
Sensitivity: 0.1911745441596188
Specificity: 0.8768382297166527
AUC-ROC: 0.8739168611804785
MCC: 0.24116329542713671
Precision: 0.4612621619467808
F1 Score: 0.19898917035829505

[INFO] Fold 3 / 10 for Random Forest


...

Accuracy: 0.6953046953046953
Sensitivity: 0.19155521263163502
Specificity: 0.8772511178070018
AUC-ROC: 0.8683195078594677
MCC: 0.24375731021645256
Precision: 0.47390984343698034
F1 Score: 0.19999857379170147

[INFO] Fold 9 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6963036963036963
Sensitivity: 0.20082484800421158
Specificity: 0.8771283863576392
AUC-ROC: 0.861103796498556
MCC: 0.24739294412138957
Precision: 0.47799938157081007
F1 Score: 0.2173484626956521

[INFO] Fold 10 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6993006993006993
Sensitivity: 0.1992890647741394
Specificity: 0.8785307291070826
AUC-ROC: 0.874506792888233
MCC: 0.2630334998070541
Precision: 0.43357706719051264
F1 Score: 0.21163742574674024


In [16]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 16.900617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 62720
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7694610778443114
Sensitivity: 0.36819818514914465
Specificity: 0.9228651640896753
AUC-ROC: 0.9271883087282708
MCC: 0.5073584748889057
Precision: 0.6335683486577419
F1 Score: 0.4160633794998692

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 16.079478 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 62720
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7754491017964071
Sensitivity: 0.3686338786552006
Specificity: 0.9192480701721887
AUC-ROC: 0.9436920012444346
MCC: 0.5151483041832305
Precision: 0.6441853046122424
F1 Score: 0.4289299242395531

[INFO] Fold 3 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 20.227489 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 62720
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

...

Accuracy: 0.7542457542457542
Sensitivity: 0.3167384730909321
Specificity: 0.915368957866146
AUC-ROC: 0.9196029317126238
MCC: 0.4606101574835569
Precision: 0.4946207399927274
F1 Score: 0.35637561503338117

[INFO] Fold 9 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 19.919733 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 9014, number of used features: 62720
[LightGBM] [Info] Start training from score -3.419559
[LightGBM] [Info] Start training from score -2.968807
[LightGBM] [Info] Start training from score -2.209840
[LightGBM] [Info] Start training from score -4.462143
[LightGBM] [Info] Start training from score -2.196781
[LightGBM] [Info] Start training from score -0.401369
[LightGBM] [Info] Start training from score -4.262347
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7642357642357642
Sensitivity: 0.3632265980867717
Specificity: 0.9197707937697038
AUC-ROC: 0.9308871097054932
MCC: 0.48878589533350675
Precision: 0.6210843091742843
F1 Score: 0.4222778324205078

[INFO] Fold 10 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 19.045347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 9014, number of used features: 62720
[LightGBM] [Info] Start training from score -3.419559
[LightGBM] [Info] Start training from score -2.970969
[LightGBM] [Info] Start training from score -2.209840
[LightGBM] [Info] Start training from score -4.471805
[LightGBM] [Info] Start training from score -2.196781
[LightGBM] [Info] Start training from score -0.401203
[LightGBM] [Info] Start training from score -4.254504
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7722277722277723
Sensitivity: 0.3601091582834653
Specificity: 0.9218018518321394
AUC-ROC: 0.928659245785797
MCC: 0.5086280717599454
Precision: 0.5908475308157337
F1 Score: 0.4101238108776596


In [17]:
# Save results to a CSV file
results_df.to_csv('DS7_Efficient.csv', index=False)

In [18]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.800898,0.597036,0.946443,0.942651,0.606511,0.690662,0.630588,996.757812,5117.515388
1,NB,0.569846,0.435012,0.920543,0.689356,0.330414,0.325272,0.328054,2388.945312,252.612986
2,KNN,0.723614,0.344183,0.917144,0.793724,0.416858,0.496757,0.374413,2407.066406,364.483869
3,Random Forest,0.698950,0.200675,0.879119,0.860838,0.261897,0.462206,0.214148,2459.816406,1321.654642
4,LGBM,0.765450,0.353379,0.919459,0.927292,0.491727,0.602396,0.405117,14178.164062,34907.246468
